# Dashboard

Dashboards interactivos con datos de donaciones y viajes de funcionarios a Noviembre de 2024

* [Dashboard Donaciones a Funcionarios](https://lookerstudio.google.com/u/1/reporting/3b61c645-811f-4cb8-b6d5-f8067a2dae45)

* [Dashboard Viajes de Funcionarios](https://lookerstudio.google.com/u/1/reporting/d1e1a26d-9c77-494d-af42-22139e9a63be)

El resto de notebook es código para scrapear y obtener los datos que se usaron en los dashboards.

Fuente: <https://www.argentina.gob.ar/anticorrupcion/obsequiosyviajes>

# Scraping 

Scraping datos de donaciones a funcionarios y viajes de funcionarios financiados por terceros 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import pandas as pd
from selenium.webdriver.chrome.service import Service

In [ ]:
# Inicio página principal 
url = 'https://www.argentina.gob.ar/anticorrupcion/obsequiosyviajes'
driver = webdriver.Chrome() # Inicializar el driver de Chrome
# Abrir la página web
driver.get(url)

# Listas para almacenar la información
descripcion = []
funcionario = []
cargo = []
organismo = []
fecha = []
valor_estimado = []
destino = []
entregado_por = []

# Esperar a que la paginación sea visible
try:
    # Esperar que la paginación esté visible
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//ul[contains(@class, 'pagination')]")))
    # Inicializamos el número de página
    pagina_actual = 1
    
    while True:    
        # Extraer la información de la página actual
        content = driver.find_elements(By.CLASS_NAME, 'descripcion')
        for c in content:
            descripcion.append(c.text)

        content = driver.find_elements(By.CLASS_NAME, 'funcionario')
        for c in content:
            funcionario.append(c.text)    

        content = driver.find_elements(By.CLASS_NAME, 'cargo')
        for c in content:
            cargo.append(c.text)

        content = driver.find_elements(By.CLASS_NAME, 'organismo')
        for c in content:
            organismo.append(c.text)

        content = driver.find_elements(By.CLASS_NAME, 'fecha')
        for c in content:
            fecha.append(c.text)

        content = driver.find_elements(By.CLASS_NAME, 'valor_estimado')
        for c in content:
            valor_estimado.append(c.text)

        content = driver.find_elements(By.CLASS_NAME, 'destino')
        for c in content:
            destino.append(c.text)

        content = driver.find_elements(By.CLASS_NAME, 'entregado_por')
        for c in content:
            entregado_por.append(c.text)

        # Intentar hacer clic en el enlace de la siguiente página
        try:
            # Buscar el enlace de la siguiente página (el enlace de "siguiente")
            pagina_actual += 1  # Incrementamos el número de página
            pagina_enlace = driver.find_element(By.XPATH, f"//ul[contains(@class, 'pagination')]//a[text()='{pagina_actual}']")
            # Si existe, clickeamos en él y pasamos a la siguiente página
            ActionChains(driver).click(pagina_enlace).perform()
            # Esperar que la paginación esté visible
            sleep(1)  # Esperar un poco para que la página cargue
    
        except Exception as e:
            print(f"No se pudo encontrar el enlace de la siguiente página. Puede que sea la última página. Página actual: {pagina_actual}")
            break  # Si no se encuentra el enlace de la siguiente página, terminamos el bucle

finally:
    # Cerramos el navegador después de completar el proceso
    driver.quit()


No se pudo encontrar el enlace de la siguiente página. Puede que sea la última página. Página actual: 414


414 páginas


In [69]:
print(len(descripcion))
print(len(funcionario))
print(len(cargo))
print(len(organismo))
print(len(fecha))
print(len(valor_estimado))
print(len(destino))
print(len(entregado_por))


8250
16510
16510
16510
16510
8250
8250
8250


Entre pagina y pagina suma valores vacios en columnas 

* funcionario

* cargo

* organismo

* fecha

In [ ]:
funcionario_clean = list(filter(lambda el: len(el) > 1, funcionario))
cargo_clean = list(filter(lambda el: len(el) > 1, cargo))
organismo_clean = list(filter(lambda el: len(el) > 1, organismo))
fecha_clean = list(filter(lambda el: len(el) > 1, fecha))
print(len(funcionario_clean))
print(len(cargo_clean))
print(len(organismo_clean))
print(len(fecha_clean))
  

8250
8250
8242
8250


**8 valores que aparecen vacios en organismo que desincroniza las listas**

**Comparar elemento por elemento para identificar posiciones desincronizadas**


In [ ]:
for i in range(len(organismo)):
    if len(organismo[i].strip()) == 0 and len(funcionario[i].strip()) != 0:
        print(f"Desincronización en el índice {i}: Organismo vacío, pero hay datos en 'funcionario'.")
        print(f"Funcionario: {funcionario[i]}, Organismo: '{organismo[i]}', Fecha: {fecha[i]}")

Desincronización en el índice 5419: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Sehtman Alejandro, Organismo: '', Fecha: 07/09/2021
Desincronización en el índice 5617: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Bauer Tristán, Organismo: '', Fecha: 04/08/2021
Desincronización en el índice 5849: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Cabandié Juan, Organismo: '', Fecha: 17/08/2021
Desincronización en el índice 5893: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Sehtman Alejandro, Organismo: '', Fecha: 23/07/2021
Desincronización en el índice 5895: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Salvarezza Roberto Carlos, Organismo: '', Fecha: 09/08/2021
Desincronización en el índice 5968: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Cabandié Juan, Organismo: '', Fecha: 11/08/2021
Desincronización en el índice 6162: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario:

**Detectar los índices desincronizados y reemplazar los valores en 'organismo'**


In [ ]:
for i in range(len(organismo)):
    if len(organismo[i].strip()) == 0 and len(funcionario[i].strip()) != 0:
        print(f"Reemplazando valor vacío en el índice {i}.")
        organismo[i] = "N/D"


Reemplazando valor vacío en el índice 5419.
Reemplazando valor vacío en el índice 5617.
Reemplazando valor vacío en el índice 5849.
Reemplazando valor vacío en el índice 5893.
Reemplazando valor vacío en el índice 5895.
Reemplazando valor vacío en el índice 5968.
Reemplazando valor vacío en el índice 6162.
Reemplazando valor vacío en el índice 9138.


**Vuelvo a generar listas clean**

In [ ]:
funcionario_clean = list(filter(lambda el: len(el) > 1, funcionario))
cargo_clean = list(filter(lambda el: len(el) > 1, cargo))
organismo_clean = list(filter(lambda el: len(el) > 1, organismo))
fecha_clean = list(filter(lambda el: len(el) > 1, fecha))

print(len(funcionario_clean))
print(len(cargo_clean))
print(len(organismo_clean))
print(len(fecha_clean))

8250
8250
8250
8250


In [ ]:
data = pd.DataFrame(list(zip(descripcion,funcionario_clean,cargo_clean,organismo_clean,fecha_clean,valor_estimado,destino,entregado_por)),
             columns =['Descripcion', 'Funcionario',"Cargo","Organismo","Fecha","Valor_estimado","Destino","Entregado_por"])
data.head(10)

,Descripcion,Funcionario,Cargo,Organismo,Fecha,Valor_estimado,Destino,Entregado_por
0,Obsequio de Cortesía: Set botella y vasos de c...,Bullrich Patricia,MINISTRA,MINISTERIO DE SEGURIDAD,22/11/2024,Inferior a 4 módulos.,,Rollé Alejandro
1,Obsequio de Cortesía: Cuadro - Servicio Penite...,Bullrich Patricia,MINISTRA,MINISTERIO DE SEGURIDAD,22/11/2024,Inferior a 4 módulos.,,Servicio Penitenciario Federal
2,"Un (1) libro: ""Volver a ser grandes""",Milei Karina Elizabeth,Secretaria Gral de Presidencia,Secretaría General,07/11/2024,Inferior a 4 módulos.,Casa de Gobierno,Dominguez Federico
3,Una (1) Letra de canción dedicada al Pte. Javi...,Milei Javier Gerardo,Presidente de la Nación,Presidencia de la Nación,06/11/2024,Inferior a 4 módulos.,Residencia Presidencial de Olivos,Storaccio Fernando Sergio
4,Una (1) Estatuilla conmemorativa por el 203° A...,Milei Karina Elizabeth,Secretaria Gral de Presidencia,Secretaría General,01/11/2024,Inferior a 4 módulos.,Casa de Gobierno,Argentina Policia Federal
5,Placa,Francos Guillermo Alberto,Jefe de Gabinete de Ministros,Jefatura de Gabinete de Ministros,19/08/2024,Inferior a 4 módulos.,,Asociación sin fines de lucro
6,Libro Universidad Católica de Santa Fe - Pasad...,Francos Guillermo Alberto,Jefe de Gabinete de Ministros,Jefatura de Gabinete de Ministros,03/10/2024,Inferior a 4 módulos.,,Universidad
7,Caja azul con lapicera,Francos Guillermo Alberto,Jefe de Gabinete de Ministros,Jefatura de Gabinete de Ministros,23/09/2024,Inferior a 4 módulos.,,Embajada de Arabia Saudita en Argentina
8,"Un (1) libro: ""La Argentina en un nuevo mundo""",Milei Javier Gerardo,Presidente de la Nación,Presidencia de la Nación,31/10/2024,Inferior a 4 módulos.,Residencia Presidencial de Olivos,la Balze Felipe de
9,"Un (1) libro: ""La Argentina en un nuevo mundo""",Milei Karina Elizabeth,Secretaria Gral de Presidencia,Secretaría General,31/10/2024,Inferior a 4 módulos.,Casa de Gobierno,la Balze Felipe de


## Convertir fechas a datetime

hay una fecha con error: 06/07/0247	debe ser error 06/07/2024

In [117]:
data['Fecha_date'] = pd.to_datetime(data['Fecha'], errors='coerce', format='%d/%m/%Y')

# Ver las filas que contienen NaT (Not a Time), es decir, las fechas mal formateadas o fuera de rango
data[data['Fecha_date'].isna()]

,Descripcion,Funcionario,Cargo,Organismo,Fecha,Valor_estimado,Destino,Entregado_por,Fecha_date
245,"Un (1) Cuadro conmemorativo ""Ordem do Mérito L...",Milei Javier Gerardo,Presidente de la Nación,Presidencia de la Nación,06/07/0247,Inferior a 4 módulos.,Residencia Presidencial de Olivos,Nacional Partido Conservador José Carlos Barna...,NaT


In [ ]:
data['Fecha'] = data['Fecha'].replace('06/07/0247', '06/07/2024')
from datetime import datetime
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%d/%m/%Y')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8250 entries, 0 to 8249
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Descripcion     8250 non-null   object        
 1   Funcionario     8250 non-null   object        
 2   Cargo           8250 non-null   object        
 3   Organismo       8250 non-null   object        
 4   Fecha           8250 non-null   datetime64[ns]
 5   Valor_estimado  8250 non-null   object        
 6   Destino         8250 non-null   object        
 7   Entregado_por   8250 non-null   object        
 8   Fecha_date      8249 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(7)
memory usage: 580.2+ KB


In [122]:
data.head(10)

,Descripcion,Funcionario,Cargo,Organismo,Fecha,Valor_estimado,Destino,Entregado_por,Fecha_date
0,Obsequio de Cortesía: Set botella y vasos de c...,Bullrich Patricia,MINISTRA,MINISTERIO DE SEGURIDAD,2024-11-22,Inferior a 4 módulos.,,Rollé Alejandro,2024-11-22
1,Obsequio de Cortesía: Cuadro - Servicio Penite...,Bullrich Patricia,MINISTRA,MINISTERIO DE SEGURIDAD,2024-11-22,Inferior a 4 módulos.,,Servicio Penitenciario Federal,2024-11-22
2,"Un (1) libro: ""Volver a ser grandes""",Milei Karina Elizabeth,Secretaria Gral de Presidencia,Secretaría General,2024-11-07,Inferior a 4 módulos.,Casa de Gobierno,Dominguez Federico,2024-11-07
3,Una (1) Letra de canción dedicada al Pte. Javi...,Milei Javier Gerardo,Presidente de la Nación,Presidencia de la Nación,2024-11-06,Inferior a 4 módulos.,Residencia Presidencial de Olivos,Storaccio Fernando Sergio,2024-11-06
4,Una (1) Estatuilla conmemorativa por el 203° A...,Milei Karina Elizabeth,Secretaria Gral de Presidencia,Secretaría General,2024-11-01,Inferior a 4 módulos.,Casa de Gobierno,Argentina Policia Federal,2024-11-01
5,Placa,Francos Guillermo Alberto,Jefe de Gabinete de Ministros,Jefatura de Gabinete de Ministros,2024-08-19,Inferior a 4 módulos.,,Asociación sin fines de lucro,2024-08-19
6,Libro Universidad Católica de Santa Fe - Pasad...,Francos Guillermo Alberto,Jefe de Gabinete de Ministros,Jefatura de Gabinete de Ministros,2024-10-03,Inferior a 4 módulos.,,Universidad,2024-10-03
7,Caja azul con lapicera,Francos Guillermo Alberto,Jefe de Gabinete de Ministros,Jefatura de Gabinete de Ministros,2024-09-23,Inferior a 4 módulos.,,Embajada de Arabia Saudita en Argentina,2024-09-23
8,"Un (1) libro: ""La Argentina en un nuevo mundo""",Milei Javier Gerardo,Presidente de la Nación,Presidencia de la Nación,2024-10-31,Inferior a 4 módulos.,Residencia Presidencial de Olivos,la Balze Felipe de,2024-10-31
9,"Un (1) libro: ""La Argentina en un nuevo mundo""",Milei Karina Elizabeth,Secretaria Gral de Presidencia,Secretaría General,2024-10-31,Inferior a 4 módulos.,Casa de Gobierno,la Balze Felipe de,2024-10-31


In [125]:
data.drop('Fecha_date',axis=1,inplace=True)
data

,Descripcion,Funcionario,Cargo,Organismo,Fecha,Valor_estimado,Destino,Entregado_por
0,Obsequio de Cortesía: Set botella y vasos de c...,Bullrich Patricia,MINISTRA,MINISTERIO DE SEGURIDAD,2024-11-22,Inferior a 4 módulos.,,Rollé Alejandro
1,Obsequio de Cortesía: Cuadro - Servicio Penite...,Bullrich Patricia,MINISTRA,MINISTERIO DE SEGURIDAD,2024-11-22,Inferior a 4 módulos.,,Servicio Penitenciario Federal
2,"Un (1) libro: ""Volver a ser grandes""",Milei Karina Elizabeth,Secretaria Gral de Presidencia,Secretaría General,2024-11-07,Inferior a 4 módulos.,Casa de Gobierno,Dominguez Federico
3,Una (1) Letra de canción dedicada al Pte. Javi...,Milei Javier Gerardo,Presidente de la Nación,Presidencia de la Nación,2024-11-06,Inferior a 4 módulos.,Residencia Presidencial de Olivos,Storaccio Fernando Sergio
4,Una (1) Estatuilla conmemorativa por el 203° A...,Milei Karina Elizabeth,Secretaria Gral de Presidencia,Secretaría General,2024-11-01,Inferior a 4 módulos.,Casa de Gobierno,Argentina Policia Federal
...,...,...,...,...,...,...,...,...
8245,Vino Marani Mukuzami,Alonso Laura,"Secretaria de Ética Pública, Transparencia y L...",Ministerio de Justicia y Derechos Humanos,2016-03-18,Inferior a 4 módulos.,,Embajada de Georgia
8246,1 taza blanca con logo institucional de la Ins...,Ramos Christian,Director,Ministerio de Modernización,2016-07-26,Inferior a 4 módulos.,,Inspección General de Justicia
8247,Cuaderno anillado promocional de la empresa de...,Kantor Mora,DIRECTORA DE PLANIFICACION DE POLITICAS DE TRA...,Ministerio de Justicia y Derechos Humanos,2017-02-10,Inferior a 4 módulos.,,HOLCIM ARGENTINA
8248,Dulce de leche 450 gs 2 unidades,Alonso Laura,"Secretaria de Ética Pública, Transparencia y L...",Ministerio de Justicia y Derechos Humanos,2016-09-13,Inferior a 4 módulos.,,Manfrey


**Guardo a csv**

In [126]:
data.to_csv('donaciones_funcionarios_2024.csv',sep=';',index=False,encoding='utf-8-sig') 

# Viajes

In [ ]:
# Inicio página principal 
url = 'https://www.argentina.gob.ar/anticorrupcion/obsequiosyviajes#edit-viajes'
driver = webdriver.Chrome() # Inicializar el driver de Chrome
# Abrir la página web
driver.get(url)

# Listas para almacenar la información
evento = []
destino = []
funcionario = []
cargo = []
organismo = []
fecha_inicio_fin = []
financiado_por = []

# Esperar a que la paginación sea visible
try:
    # Esperar que la paginación esté visible
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="edit-viajes-body"]/nav/ul/li[2]/a')))
    
    # Inicializamos el número de página
    pagina_actual = 1
    
    while True:    
        # Extraer la información de la página actual
        content = driver.find_elements(By.CLASS_NAME, 'evento')
        for c in content:
            evento.append(c.text)

        content = driver.find_elements(By.CLASS_NAME, 'pais')
        for c in content:
            destino.append(c.text)    

        content = driver.find_elements(By.CLASS_NAME, 'funcionario')
        for c in content:
            funcionario.append(c.text)

        content = driver.find_elements(By.CLASS_NAME, 'cargo')
        for c in content:
            cargo.append(c.text)

        content = driver.find_elements(By.CLASS_NAME, 'organismo')
        for c in content:
            organismo.append(c.text)

        content = driver.find_elements(By.CLASS_NAME, 'fecha')
        for c in content:
            fecha_inicio_fin.append(c.text)

        content = driver.find_elements(By.CLASS_NAME, 'financiamiento')
        for c in content:
            financiado_por.append(c.text)
        
        # Intentar hacer clic en el enlace de la siguiente página. tiene paginacion dinamica
        try:
            pagina_actual += 1  # Incrementar el número de página
            # Buscar todos los enlaces de la paginación
            enlaces_pagina = driver.find_elements(By.XPATH, '//*[@id="edit-viajes-body"]/nav/ul/li/a')
            # Verificar si el enlace con el número de página está presente
            pagina_enlace = None
            for enlace in enlaces_pagina:
                if enlace.text == str(pagina_actual):  # Buscamos el siguiente número de página
                    pagina_enlace = enlace
                    break
            
            if pagina_enlace: # si es verdadero
                # Hacer clic en el enlace de la siguiente página
                ActionChains(driver).click(pagina_enlace).perform()
                sleep(1)  # Esperar un poco para que la página cargue
            else:
                # Si no encontramos el enlace de la siguiente página, hemos llegado al final
                print(f"No se pudo encontrar el enlace de la siguiente página. Puede que sea la última página. Página actual: {pagina_actual}")
                break  # Si no se encuentra el enlace de la siguiente página, terminamos el bucle

        except Exception as e:
            print(f"Error al intentar hacer clic en la siguiente página: {e}")
            break   
        

finally:
    # Cerramos el navegador después de completar el proceso
    driver.quit()


No se pudo encontrar el enlace de la siguiente página. Puede que sea la última página. Página actual: 99


99 páginas

In [ ]:
print(len(evento))
print(len(destino))
print(len(funcionario))
print(len(cargo))
print(len(organismo))
print(len(fecha_inicio_fin))
print(len(financiado_por))

1944
1944
3904
3904
3904
3904
1944


Suma valores vacios en columnas 

* funcionario
* cargo
* organismo
* fecha_inicio_fin

In [ ]:
funcionario_clean = list(filter(lambda el: len(el) >= 1, funcionario))
cargo_clean = list(filter(lambda el: len(el) >= 1, cargo))
organismo_clean = list(filter(lambda el: len(el) >= 1, organismo))
fecha_inicio_fin_clean = list(filter(lambda el: len(el) >= 1, fecha_inicio_fin))

print(len(funcionario_clean))
print(len(cargo_clean))
print(len(organismo_clean))
print(len(fecha_inicio_fin_clean))
  

1944
1944
1936
1944


**8 valores que aparecen vacios en organismo que desincroniza las listas** 

In [ ]:
# Comparar elemento por elemento para identificar posiciones desincronizadas
for i in range(len(organismo)):
    if len(organismo[i].strip()) == 0 and len(funcionario[i].strip()) != 0:
        print(f"Desincronización en el índice {i}: Organismo vacío, pero hay datos en 'funcionario'.")
        print(f"Funcionario: {funcionario[i]}, Organismo: '{organismo[i]}', Fecha: {fecha_inicio_fin[i]}")

Desincronización en el índice 460: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Barttfeld Federico Alejandro, Organismo: '', Fecha: Desde 16/03/2024 hasta 23/03/2024
Desincronización en el índice 636: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Roig Alexandre, Organismo: '', Fecha: Desde 21/10/2023 hasta 26/10/2023
Desincronización en el índice 1260: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Schiavi Antonela, Organismo: '', Fecha: Desde 09/07/2022 hasta 02/08/2022
Desincronización en el índice 1319: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Salom Diego, Organismo: '', Fecha: Desde 08/05/2022 hasta 15/05/2022
Desincronización en el índice 1386: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Teper Marcelo, Organismo: '', Fecha: Desde 10/11/2021 hasta 19/11/2021
Desincronización en el índice 1387: Organismo vacío, pero hay datos en 'funcionario'.
Funcionario: Santuchi Gustavo Rubén, Organismo: '

**Detectar los índices desincronizados y reemplazar los valores en 'organismo'** 

In [ ]:
for i in range(len(organismo)):
    if len(organismo[i].strip()) == 0 and len(funcionario[i].strip()) != 0:
        print(f"Reemplazando valor vacío en el índice {i}.")
        organismo[i] = "N/D"


Reemplazando valor vacío en el índice 460.
Reemplazando valor vacío en el índice 636.
Reemplazando valor vacío en el índice 1260.
Reemplazando valor vacío en el índice 1319.
Reemplazando valor vacío en el índice 1386.
Reemplazando valor vacío en el índice 1387.
Reemplazando valor vacío en el índice 1794.
Reemplazando valor vacío en el índice 1988.


**Vuelvo a generar listas clean**

In [ ]:
funcionario_clean = list(filter(lambda el: len(el) >= 1, funcionario))
cargo_clean = list(filter(lambda el: len(el) >= 1, cargo))
organismo_clean = list(filter(lambda el: len(el) >= 1, organismo))
fecha_inicio_fin_clean = list(filter(lambda el: len(el) >= 1, fecha_inicio_fin))

print(len(funcionario_clean))
print(len(cargo_clean))
print(len(organismo_clean))
print(len(fecha_inicio_fin_clean))

1944
1944
1944
1944


In [ ]:
data = pd.DataFrame(list(zip(evento,destino,funcionario_clean,cargo_clean,organismo_clean,fecha_inicio_fin_clean,financiado_por)),
             columns =['evento', 'destino',"funcionario","cargo","organismo","fecha_inicio_fin","financiado_por"])

data.head(10)

,evento,destino,funcionario,cargo,organismo,fecha_inicio_fin,financiado_por
0,VII Encuentro de egresados de la Universidad d...,ARGENTINA,BARRA Rodolfo Carlos,Procurador del Tesoro de la Nación,Procuración del Tesoro de la Nación,Desde 21/11/2024 hasta 22/11/2024,Instituto para el Desarrollo Constitucional (I...
1,Reuniones de la Secretaría de Hacienda de la N...,ESTADOS UNIDOS,Mana Matías Javier,Subsecretario de Relaciones Financieras Intern...,Ministerio de Economía,Desde 12/11/2024 hasta 16/11/2024,Banco Mundial
2,Comisión General y 197a Reunión del Directorio...,BRASIL,Mana Matías Javier,Subsecretario de Relaciones Financieras Intern...,Ministerio de Economía,Desde 17/11/2024 hasta 19/11/2024,Banco de Desarrollo FONPLATA
3,Diálogo Político de Alto Nivel sobre Conectivi...,CHILE,BRUNELLI DIEGO MARTIN,ASESOR DE LA INTERVENCIÓN,Ente Nacional de Comunicaciones,Desde 04/11/2024 hasta 07/11/2024,UNIÓN EUROPEA
4,75º Aniversario de los Convenios de Ginebra de...,MÉXICO,Vilas Paula,Consejera,"Ministerio de Relaciones Exteriores, Comercio ...",Desde 03/07/2024 hasta 06/07/2024,Comité Internacional de la Cruz Roja
5,Technical Meeting on Decision-Making for Susta...,AUSTRIA,Hosid Ariel Martin,Ingeniero Nuclear,Jefatura de Gabinete de Ministros,Desde 03/11/2024 hasta 13/11/2024,Organismo Internacional de Energía Atómica (OIEA)
6,Reunión del Comité de Gestión del Consejo Cons...,SUIZA,GRAPSAS MARIA AGUSTINA,ASESORA DE LA INTERVENCIÓN,Dirección General de Asuntos Jurídicos y Regul...,Desde 29/10/2024 hasta 10/11/2024,"UNIÓN POSTAL DE LAS AMÉRICAS, ESPAÑA Y PORTUGAL"
7,Fellowship Training in Operation of Cyclotrons...,ITALIA,Rondon Brito David Enrique,Operador de ciclotrón,Comisión Nacional de Energía Atómica,Desde 12/10/2024 hasta 10/11/2024,ORGANISMO INTERNACIONAL DE ENERGIA ATOMICA
8,Capacitación en ciclotrón y radiofarmacia en e...,ITALIA,Cuevas Dalmiro Conrado,Becario,Comisión Nacional de Energía Atómica,Desde 12/10/2024 hasta 10/11/2024,OIEA
9,Reunión Intersesional de la Junta Internaciona...,GRECIA,Chomik Silvia Beatriz,Jefe de División Cartografía entrante,Servicio de Hidrografía Naval,Desde 09/11/2024 hasta 17/11/2024,Organización Hidrográfica Internacional


## Convertir fechas a datetime

valor erroneo 2924

In [ ]:
data[data['fecha_inicio_fin'].str.contains('2924')]

,evento,destino,funcionario,cargo,organismo,fecha_inicio_fin,financiado_por
28,ASISTENCIA AL ACTO Y ALMUERZO DE CONFRATERNIDAD,ARGENTINA,COLLOMB Marcelo Oscar,PRESIDENTE,MINISTERIO DE CAPITAL HUMANO,Desde 26/10/2924 hasta 27/10/2024,Federeación Provincial de Mutualidades de Córdoba


In [ ]:
data['fecha_inicio_fin'] = data['fecha_inicio_fin'].str.replace('2924', '2024')

In [ ]:
# Limpiar el texto para eliminar las palabras 'Desde' y 'hasta'
data['fecha_inicio_fin_limpia'] = data['fecha_inicio_fin'].str.replace('Desde ', '').str.replace(' hasta ', ' ')
# Separar en dos columnas: fecha de inicio y fecha de fin
data[['fecha_inicio', 'fecha_fin']] = data['fecha_inicio_fin_limpia'].str.split(' ', expand=True)
# Convertir las nuevas columnas en tipo de dato datetime
data['fecha_inicio'] = pd.to_datetime(data['fecha_inicio'], format='%d/%m/%Y')
data['fecha_fin'] = pd.to_datetime(data['fecha_fin'], format='%d/%m/%Y')
data

,evento,destino,funcionario,cargo,organismo,fecha_inicio_fin,financiado_por,fecha_inicio_fin_limpia,fecha_inicio,fecha_fin
0,VII Encuentro de egresados de la Universidad d...,ARGENTINA,BARRA Rodolfo Carlos,Procurador del Tesoro de la Nación,Procuración del Tesoro de la Nación,Desde 21/11/2024 hasta 22/11/2024,Instituto para el Desarrollo Constitucional (I...,21/11/2024 22/11/2024,2024-11-21,2024-11-22
1,Reuniones de la Secretaría de Hacienda de la N...,ESTADOS UNIDOS,Mana Matías Javier,Subsecretario de Relaciones Financieras Intern...,Ministerio de Economía,Desde 12/11/2024 hasta 16/11/2024,Banco Mundial,12/11/2024 16/11/2024,2024-11-12,2024-11-16
2,Comisión General y 197a Reunión del Directorio...,BRASIL,Mana Matías Javier,Subsecretario de Relaciones Financieras Intern...,Ministerio de Economía,Desde 17/11/2024 hasta 19/11/2024,Banco de Desarrollo FONPLATA,17/11/2024 19/11/2024,2024-11-17,2024-11-19
3,Diálogo Político de Alto Nivel sobre Conectivi...,CHILE,BRUNELLI DIEGO MARTIN,ASESOR DE LA INTERVENCIÓN,Ente Nacional de Comunicaciones,Desde 04/11/2024 hasta 07/11/2024,UNIÓN EUROPEA,04/11/2024 07/11/2024,2024-11-04,2024-11-07
4,75º Aniversario de los Convenios de Ginebra de...,MÉXICO,Vilas Paula,Consejera,"Ministerio de Relaciones Exteriores, Comercio ...",Desde 03/07/2024 hasta 06/07/2024,Comité Internacional de la Cruz Roja,03/07/2024 06/07/2024,2024-07-03,2024-07-06
...,...,...,...,...,...,...,...,...,...,...
1939,Secondary Expert Group Meeting - Educationfor ...,AUSTRIA,Escuder Maria Laura,Asesor,Ministerio de Justicia y Derechos Humanos,Desde 11/02/2017 hasta 17/02/2017,UNOCD,11/02/2017 17/02/2017,2017-02-11,2017-02-17
1940,ENCUENTRO REGIONAL LUCHA CONTRA EL DELITO TRAN...,COLOMBIA,RAMIREZ JUAN CARLOS,PREFECTO MAYOR/JEFE DE LA CENTRAL DE INTELIGEN...,Prefectura Naval Argentina,Desde 06/03/2023 hasta 10/03/2023,Programa de Cooperación Internacional EUROFRONT,06/03/2023 10/03/2023,2023-03-06,2023-03-10
1941,Primera Cumbre latinoamericana de Comunicación...,PERÚ,Alonso Laura,"Secretaria de Etica Publica, Transparencia y L...",Ministerio de Justicia y Derechos Humanos,Desde 13/07/2016 hasta 16/07/2016,Instituto de Comunicacion Política y Gestión P...,13/07/2016 16/07/2016,2016-07-13,2016-07-16
1942,II Simposio Internacional de Aguas Transfronte...,BOLIVIA,Rebok Nicolás Sergio,Director,"Ministerio de Relaciones Exteriores, Comercio ...",Desde 29/11/2022 hasta 04/12/2022,Comité Intergubernamental Coordinador de los P...,29/11/2022 04/12/2022,2022-11-29,2022-12-04


In [ ]:
data.drop(['fecha_inicio_fin','fecha_inicio_fin_limpia'],axis=1,inplace=True)
data

,evento,destino,funcionario,cargo,organismo,financiado_por,fecha_inicio,fecha_fin
0,VII Encuentro de egresados de la Universidad d...,ARGENTINA,BARRA Rodolfo Carlos,Procurador del Tesoro de la Nación,Procuración del Tesoro de la Nación,Instituto para el Desarrollo Constitucional (I...,2024-11-21,2024-11-22
1,Reuniones de la Secretaría de Hacienda de la N...,ESTADOS UNIDOS,Mana Matías Javier,Subsecretario de Relaciones Financieras Intern...,Ministerio de Economía,Banco Mundial,2024-11-12,2024-11-16
2,Comisión General y 197a Reunión del Directorio...,BRASIL,Mana Matías Javier,Subsecretario de Relaciones Financieras Intern...,Ministerio de Economía,Banco de Desarrollo FONPLATA,2024-11-17,2024-11-19
3,Diálogo Político de Alto Nivel sobre Conectivi...,CHILE,BRUNELLI DIEGO MARTIN,ASESOR DE LA INTERVENCIÓN,Ente Nacional de Comunicaciones,UNIÓN EUROPEA,2024-11-04,2024-11-07
4,75º Aniversario de los Convenios de Ginebra de...,MÉXICO,Vilas Paula,Consejera,"Ministerio de Relaciones Exteriores, Comercio ...",Comité Internacional de la Cruz Roja,2024-07-03,2024-07-06
...,...,...,...,...,...,...,...,...
1939,Secondary Expert Group Meeting - Educationfor ...,AUSTRIA,Escuder Maria Laura,Asesor,Ministerio de Justicia y Derechos Humanos,UNOCD,2017-02-11,2017-02-17
1940,ENCUENTRO REGIONAL LUCHA CONTRA EL DELITO TRAN...,COLOMBIA,RAMIREZ JUAN CARLOS,PREFECTO MAYOR/JEFE DE LA CENTRAL DE INTELIGEN...,Prefectura Naval Argentina,Programa de Cooperación Internacional EUROFRONT,2023-03-06,2023-03-10
1941,Primera Cumbre latinoamericana de Comunicación...,PERÚ,Alonso Laura,"Secretaria de Etica Publica, Transparencia y L...",Ministerio de Justicia y Derechos Humanos,Instituto de Comunicacion Política y Gestión P...,2016-07-13,2016-07-16
1942,II Simposio Internacional de Aguas Transfronte...,BOLIVIA,Rebok Nicolás Sergio,Director,"Ministerio de Relaciones Exteriores, Comercio ...",Comité Intergubernamental Coordinador de los P...,2022-11-29,2022-12-04


**Guardo a csv**

In [ ]:
data.to_csv('viajes_funcionarios_2024.csv',sep=';',index=False,encoding='utf-8-sig') 